In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 8.8 MB/s eta 0:00:00


In [2]:
import os
from kaggle_secrets import UserSecretsClient

from IPython.display import Markdown

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages

class TravelBookingState(TypedDict):
    """State representing the user's travel booking conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes.
    messages: Annotated[list, add_messages]

    # The user's current booking preferences.
    booking_request: dict

    # Flag indicating that the booking is completed.
    finished: bool


# System instruction for the Travel Booking Assistant
TRAVELBOT_SYSINT = (
    "system",
    "You are a TravelBookingBot, an intelligent travel assistant for Sri Lanka. A human will talk to you "
    "about planning trips, and you will help them by recommending, confirming, and simulating bookings. "
    "The user can request customized travel plans, such as 'Book me a 3-day surf trip in Arugam Bay under $300.' "
    "\n\n"
    "When the user makes a request:\n"
    "- Break it down to identify destination, duration, activity type, and budget.\n"
    "- Use retrieve_packages to search available travel options (or dummy data).\n"
    "- Match options based on user preferences and present top choices.\n"
    "- Confirm the trip details with the user before simulating booking using book_trip.\n\n"
    "You can also clear a request with clear_booking, or retrieve the current status with get_booking_request.\n"
    "Be polite, helpful, and focused only on Sri Lankan travel-related questions.\n\n"
    "Once the user confirms, simulate the booking using book_trip, thank them, and finish the conversation.\n"
    "If any functions aren't implemented yet, inform the user they're still under development."
)

# Initial greeting message for the travel assistant
WELCOME_MSG = "Ayubowan! I’m your Sri Lankan Travel Buddy. Type `q` to quit. How can I help you plan your next adventure?"


In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Use Gemini 2.0 flash model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# --- REPLACE: chatbot and graph setup for travel booking ---

def travel_chatbot(state: TravelBookingState) -> TravelBookingState:
    """Travel chatbot using Gemini to handle booking conversations."""
    message_history = [TRAVELBOT_SYSINT] + state["messages"]
    response = llm.invoke(message_history)
    return {
        "messages": [response],
        "booking_request": state.get("booking_request", {}),
        "finished": False
    }

# Set up the LangGraph using the new travel booking state
graph_builder = StateGraph(TravelBookingState)

# Add the chatbot node (renamed for clarity)
graph_builder.add_node("travel_chatbot", travel_chatbot)

# Define entrypoint and basic flow
graph_builder.set_entry_point("travel_chatbot")
graph_builder.add_edge(START, "travel_chatbot")

# This could lead to another node or loop, but we'll end it here for now
graph_builder.add_edge("travel_chatbot", END)

# Compile the graph
travel_chat_graph = graph_builder.compile()


In [5]:
#from IPython.display import Image, display

#Image(travel_chat_graph.get_graph().draw_mermaid_png())


In [6]:
from pprint import pprint

user_msg = "Hello, what can you do?"
state = travel_chat_graph.invoke({
    "messages": [user_msg],
    "booking_request": {},
    "finished": False
})

# Uncomment this to inspect the full state
# pprint(state)

# Print chat history (user + AI response)
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")



HumanMessage: Hello, what can you do?
AIMessage: Hello! I'm your personal travel assistant for Sri Lanka. I can help you plan and even simulate booking your dream trip. Just tell me what you're looking for, whether it's a relaxing beach vacation, an adventurous cultural tour, or anything in between. I can provide recommendations based on your preferences, including destination, duration, activity type, and budget. How can I help you plan your Sri Lankan adventure today?


In [7]:
user_msg = "Can you plan me a weekend hiking trip in Ella under 200 dollars?"


# Append new user message to previous message history
state["messages"].append(user_msg)

# Invoke the travel booking assistant again with updated state
state = travel_chat_graph.invoke(state)

# Print conversation history
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")
    


HumanMessage: Hello, what can you do?
AIMessage: Hello! I'm your personal travel assistant for Sri Lanka. I can help you plan and even simulate booking your dream trip. Just tell me what you're looking for, whether it's a relaxing beach vacation, an adventurous cultural tour, or anything in between. I can provide recommendations based on your preferences, including destination, duration, activity type, and budget. How can I help you plan your Sri Lankan adventure today?
HumanMessage: Can you plan me a weekend hiking trip in Ella under 200 dollars?
AIMessage: Okay, I can help you with that! You're looking for a weekend hiking trip in Ella, with a budget under $200.

Let me quickly retrieve some suitable packages for you.

```tool_code
print(travel_bot.retrieve_packages(destination='Ella', duration=2, activity_type='hiking', budget=200))
```


In [8]:
from langchain_core.messages.ai import AIMessage

def human_node(state: TravelBookingState) -> TravelBookingState:
    """Displays model message to user, takes user input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


def travel_chatbot_with_welcome(state: TravelBookingState) -> TravelBookingState:
    """Chatbot entry that shows welcome message if first turn, else continues conversation."""
    if state["messages"]:
        new_output = llm.invoke([TRAVELBOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}


# Build the new looped graph
graph_builder = StateGraph(TravelBookingState)

# Add chatbot and human interaction nodes
graph_builder.add_node("travel_chatbot", travel_chatbot_with_welcome)
graph_builder.add_node("human", human_node)

# Start at chatbot, then always go to human, and repeat
graph_builder.add_edge(START, "travel_chatbot")
graph_builder.add_edge("travel_chatbot", "human")
graph_builder.add_edge("human", "travel_chatbot")  # loop continues

# Define how to end
#graph_builder.set_finish_condition(lambda state: state.get("finished") is True)

# Compile the final travel chat loop
travel_loop_graph = graph_builder.compile()


In [9]:
from typing import Literal

def maybe_exit_human_node(state: TravelBookingState) -> Literal["travel_chatbot", "__end__"]:
    """Route to chatbot or end depending on user exit intent."""
    if state.get("finished", False):
        return END
    else:
        return "travel_chatbot"

graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Compile graph with conditional edge
chat_with_human_graph = graph_builder.compile()

# Visualize the conversation flow
#Image(chat_with_human_graph.get_graph().draw_mermaid_png())


In [10]:
from langchain_core.tools import tool

@tool
def get_packages(destination: str, max_budget: int) -> str:
    """Retrieve available travel packages for a given destination and budget."""

    # Dummy packages database (could be from a real DB in future)
    packages = {
        "arugam bay": [
            {"title": "Surf & Chill 3-Day Tour", "price": 280, "details": "Includes surf lessons, 2 nights hotel, and transport."},
            {"title": "Budget Surf Hostel Experience", "price": 190, "details": "Shared hostel, surfboard rental, and beach BBQ."},
            {"title": "Luxury Surf Retreat", "price": 450, "details": "4-star hotel, private instructor, meals included."}
        ],
        "ella": [
            {"title": "Ella Hiking Adventure", "price": 150, "details": "2 nights guesthouse, Ella Rock and 9 Arch Hike."},
            {"title": "Luxury Mountain Getaway", "price": 300, "details": "3 nights boutique stay, guided treks, breakfast included."}
        ]
    }

    # Normalize destination
    dest = destination.lower()

    if dest not in packages:
        return f"Sorry, we currently have no packages listed for {destination.title()}."

    # Filter packages under budget
    matching = [pkg for pkg in packages[dest] if pkg["price"] <= max_budget]

    if not matching:
        return f"No available packages under ${max_budget} for {destination.title()}."

    # Format result
    result = f"Packages available for {destination.title()} under ${max_budget}:\n"
    for pkg in matching:
        result += f"\n- {pkg['title']} (${pkg['price']}): {pkg['details']}"

    return result


In [11]:
from langgraph.prebuilt import ToolNode

# Define the tools (get_packages instead of get_menu)
tools = [get_packages]
tool_node = ToolNode(tools)

# Bind the tool to the model
llm_with_tools = llm.bind_tools(tools)

def maybe_route_to_tools(state: TravelBookingState) -> Literal["tools", "human"]:
    """Route between human or tool nodes, depending on tool calls."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"

def travel_chatbot_with_tools(state: TravelBookingState) -> TravelBookingState:
    """Chatbot that interacts with tools (get travel packages)."""
    defaults = {"booking_request": {}, "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([TRAVELBOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}

# Build the travel booking conversation graph
graph_builder = StateGraph(TravelBookingState)

# Add the travel chatbot, human, and tools nodes
graph_builder.add_node("travel_chatbot", travel_chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)

# Conditional routing
graph_builder.add_conditional_edges("travel_chatbot", maybe_route_to_tools)
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools always route back to the chatbot
graph_builder.add_edge("tools", "travel_chatbot")

# Start at the chatbot node
graph_builder.add_edge(START, "travel_chatbot")

# Compile the graph
travel_graph_with_tools = graph_builder.compile()

# Visualize the flow
#Image(travel_graph_with_tools.get_graph().draw_mermaid_png())


In [12]:
# The default recursion limit for traversing nodes is 25 - setting it higher means
# you can try a more complex order with multiple steps and round-trips (and you
# can chat for longer!)
config = {"recursion_limit": 100}

# Remember that this will loop forever, unless you input `q`, `quit` or one of the
# other exit terms defined in `human_node`.
# Uncomment this line to execute the graph:
# Final Execution (with tool-based graph)
state = travel_graph_with_tools.invoke({"messages": []}, config)


# Things to try:
#  - Just chat! There's no ordering or menu yet.
#  - 'q' to exit.


# pprint(state)

Model: Ayubowan! I’m your Sri Lankan Travel Buddy. Type `q` to quit. How can I help you plan your next adventure?


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.